In [1]:
import copy
import os

import pandas as pd
import polib

# Monkey translate strings to know where they can be translated, and where not.

In [2]:
# pofile = polib.pofile('./LC_MESSAGES/crawl.po')

# for entry in pofile.untranslated_entries():
# #     if entry.msgstr.startswith('***') and entry.msgstr.endswith('***'):
#         entry.msgstr = f"^^^{entry.msgid}$$$"
    
# pofile.save()

In [3]:
os.system('cd ../../../; xgettext -k_:1 -k__:1c,2 -kpgettext_noop:1c,2 -o ./lang/po/mon-data.po ./mon-data.h')
monsters_pofile = polib.pofile('../../po/mon-data.po')
translations_pofile = polib.pofile('../../po/ru/LC_MESSAGES/crawl.po')

# Add all monsters to appropriate contexts for correct translation

In [4]:
ARTICLE_A = "a"
ARTICLE_THE = "the "  # note the trailing space

monsters_contexts = [
    ("", ARTICLE_A),
    ("slain by", ARTICLE_A),
    ("The mighty blow completely misses %s!", ""),
    ("Your bite misses %s.", ARTICLE_THE),
    ("%s hits a putrid demon", ARTICLE_THE),
    ("The giant cockroach hits %s", ARTICLE_A),
]
vowels = 'aeiou'


def new_entries_gen():
    for entry in monsters_pofile:
        if not entry.msgid:
            continue
        for context, article in monsters_contexts:
            new_entry = copy.copy(entry)
            
            if context:
                new_entry.msgctxt = context
            
            if article == ARTICLE_A:
                if new_entry.msgid[0] in vowels:
                    article = 'an '
                else:
                    article = 'a '

            new_entry.msgid = f'{article}{new_entry.msgid}'
            yield new_entry


# for entry in list(new_entries_gen()):  # without list we get infinite loop, because we add to list we're iterating on
#     monsters_pofile.append(entry)
    
# monsters_pofile.save()

# Вывести список существ

In [6]:
def get_monster_ids(monsters_pofile):
    for monster_entry in monsters_pofile:
        yield(monster_entry.msgid)
        

def get_monster_ids_and_translations(monster_ids, translations_pofile):
    data = dict(map(lambda mid: (mid, ""), monster_ids))
    
    for entry in translations_pofile:
        msgid = entry.msgid
        
        # normalize msgid
        if msgid.startswith("a "):
            startindex = 2
        elif msgid.startswith("an "):
            startindex = 3
        elif msgid.startswith("the "):
            startindex = 4
        else:
            startindex = 0

        msgid = msgid[startindex:]

        if msgid in data:
            if entry.msgctxt:
                print(f"Skipping ({msgid}): txn in context [{entry.msgctxt}] == <{entry.msgstr}>.")
                continue
            data[msgid] = entry.msgstr
    
    return data
    

df = pd.DataFrame.from_records(
    list(
        get_monster_ids_and_translations(
            get_monster_ids(
                monsters_pofile
            ),
            translations_pofile,
        ).items()
        )
)

Skipping (azure jelly): txn in context [slain by] == <лазурным желе>.
Skipping (wood golem): txn in context [slain by] == <деревянным големом>.
Skipping (spirit): txn in context [slain by] == <духом>.
Skipping (paladin): txn in context [slain by] == <паладином>.
Skipping (pan): txn in context [slain by] == <кастрюлей>.
Skipping (lamia): txn in context [slain by] == <ламией>.
Skipping (chaos butterfly): txn in context [slain by] == <бабочкой хаоса>.
Skipping (rock worm): txn in context [slain by] == <каменным червём>.
Skipping (lava fish): txn in context [slain by] == <лавовой рыбой>.
Skipping (clay golem): txn in context [slain by] == <глиняным големом>.
Skipping (stone golem): txn in context [slain by] == <каменным големом>.
Skipping (vapour): txn in context [slain by] == <паром>.
Skipping (lava worm): txn in context [slain by] == <лавовым червём>.
Skipping (deep elf priest): txn in context [slain by] == <глубинным эльфом жрецом>.
Skipping (putrid demonspawn): txn in context [slain by

In [7]:
df.to_clipboard()